In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity


In [2]:
df = pd.read_csv('data\dataset.csv')
df.shape

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\dimaag\AppData\Local\Temp\ipykernel_34404\2851665957.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('data\dataset.csv')


(114000, 21)

In [3]:
df_final = df.copy()
col = []
for i in df.columns:
  if df[i].dtype == 'object':
    col.append(i)
print(col)

['track_id', 'artists', 'album_name', 'track_name', 'track_genre']


In [4]:
df_final_1 = df[col]
df_final_1

,track_id,artists,album_name,track_name,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,acoustic
...,...,...,...,...,...
113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,world-music
113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,world-music
113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,world-music
113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,world-music


In [5]:
df_final_1.dropna(inplace=True)
df_final_1.reset_index(drop=True,inplace=True)
df_final_1.isnull().sum()
# df_final_1 = df_final_1.sample(5000)
df_final_1.shape

C:\Users\dimaag\AppData\Local\Temp\ipykernel_34404\862107638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_1.dropna(inplace=True)


(113999, 5)

In [6]:
df_final_1.drop(columns=['track_id'],inplace=True)


df_final_1.shape


C:\Users\dimaag\AppData\Local\Temp\ipykernel_34404\937831077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_1.drop(columns=['track_id'],inplace=True)


(113999, 4)

In [7]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_final_1['track_name'])
tfidf_matrix.shape


(113999, 48415)

In [10]:
from scipy.sparse import vstack

batch_size = 20
results = []

for start in range(0, tfidf_matrix.shape[0], batch_size):
    end = min(start + batch_size, tfidf_matrix.shape[0])
    batch = tfidf_matrix[start:end]
    
    # sim = cosine_similarity(batch, tfidf_matrix)  # shape: (batch_size, full)
    sim = linear_kernel(batch, tfidf_matrix)  # shape: (batch_size, full)
    results.append(sim)


MemoryError: Unable to allocate 17.4 MiB for an array with shape (20, 113999) and data type float64

In [ ]:
df[df['track_name'] == 'Change Your World'].index[0] # Get the index of the row with 'Water Into Light'

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
def get_recommendations(title, cosine_sim=similar):
    idx = df[df['track_name'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    track_name =[]
    for i in sim_scores[1:5]:
        track_name.append(df.iloc[i[0]].track_name)
    return track_name

In [ ]:
get_recommendations("Can't Help Falling In Love",cosine_sim=similar)  # Get recommendations for 'Water Into Light'

['Comedy', 'Ghost - Acoustic', 'To Begin Again', 'Hold On']